In [ ]:
import os
import logging
logging.basicConfig(level=logging.INFO)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import tensorflow_hub as hub
from bert_serving.client import BertClient
bc = BertClient()

In [ ]:
import datahelper
datahelper = datahelper.DataHelper(embedding_path="../embedding/STCWiki/STCWiki_mincount0.model.bin")

In [ ]:
REMOVE_STOPWORDS = False
TO_LOWER = True
TOKEN_TYPE = 'nltk'
EMB = 'stc' # glove or stc

In [ ]:
trainX, trainX_bert, trainND, trainDQ, train_turns, train_masks = datahelper.get_model_train_data(
    'train',
    TOKEN_TYPE, 
    REMOVE_STOPWORDS, 
    TO_LOWER,
    EMB,
    bert=True,
)

devX, devX_bert, devND, devDQ, dev_turns, dev_masks = datahelper.get_model_train_data(
    'dev',
    TOKEN_TYPE, 
    REMOVE_STOPWORDS, 
    TO_LOWER,
    EMB,
    bert=True,
)

testX, testX_bert, test_turns, test_masks = datahelper.get_model_test_data(
    TOKEN_TYPE, 
    REMOVE_STOPWORDS, 
    TO_LOWER,
    EMB,
    bert=True,
)

In [ ]:
trainX.shape, trainX_bert.shape

In [ ]:
devX.shape, devX_bert.shape

In [ ]:
import pickle
pickle.dump(trainX_bert, open('trainX_bert_512.p', 'wb'))
pickle.dump(devX_bert, open('devX_bert_512.p', 'wb'))
pickle.dump(testX_bert, open('testX_bert_512.p', 'wb'))

In [ ]:
trainX = pickle.load(open('trainX_bert.p', 'rb'))
devX = pickle.load(open('devX_bert.p', 'rb'))
testX = pickle.load(open('testX_bert.p', 'rb'))

In [ ]:
devX_berts.shape, devX_bert.shape

### TensorFlow Hub

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [4]:
def get_sess_config():
    sess_config = tf.ConfigProto(allow_soft_placement=True)
    sess_config.gpu_options.allow_growth = True
    return sess_config

def create_tokenizer_from_hub_module(bert_module, sess_config=get_sess_config()):
    """Get the vocab file and casing info from the Hub module."""
    bert_hub_model_handle = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
    with tf.Graph().as_default():
        bert_module = hub.Module(bert_hub_model_handle, trainable=True)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session(config=sess_config) as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                                  tokenization_info["do_lower_case"]])
    return tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

In [5]:
bert_module = get_bert_module()
tokenizer = create_tokenizer_from_hub_module(bert_module)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [31]:
# Get input_id
sent = 'i whaat wechatt no'
bert_tokens = []
bert_tokens.append("[CLS]")
bert_tokens.extend(tokenizer.tokenize(sent))
bert_tokens.append("[SEP]")
input_ids = tokenizer.convert_tokens_to_ids(bert_tokens)

In [32]:
bert_tokens

['[CLS]', 'i', 'w', '##ha', '##at', 'we', '##cha', '##tt', 'no', '[SEP]']

In [33]:
input_ids

[101, 1045, 1059, 3270, 4017, 2057, 7507, 4779, 2053, 102]

In [ ]:
bert_inputs = dict(
    input_ids=input_ids,
)

bert_outputs = bert_module(
    signature="tokens", 
    as_dict=True,
)